In [1]:
import pandas as pd
train_df = pd.read_csv("../sales_train.csv.gz")
test_df  = pd.read_csv("../test.csv.gz")

In [2]:
%store -r __prepare_data

In [3]:
__prepare_data

In [4]:
all_data = all_data[all_data.date_block_num >= 12] # no lags before time 0
all_data.fillna(0, inplace=True)
print(all_data.isnull().values.any())

False


In [5]:
from sklearn.model_selection import train_test_split

target = all_data.target.values
target[target>20] = 20

predictors = ["target_lag_1", "target_lag_2", "target_lag_3", "target_lag_6", "target_lag_12",
              "item_target_enc_lag_1","shop_target_enc_lag_1",
              "item_target_enc_ave",  "shop_target_enc_ave"]

X_train, X_test, y_train, y_test = \
    train_test_split(all_data[predictors], target, test_size=0.2, random_state=321)

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model = RandomForestRegressor(n_estimators=32, max_features=3, n_jobs=6) #len(predictors)/2
model.fit(X_train, y_train)
ypred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, ypred))
print("RMSE: %f" % (rmse))

RMSE: 0.740057


In [7]:
full_model = RandomForestRegressor(n_estimators=128, max_features=3, n_jobs=6) #len(predictors)/2
full_model.fit(all_data[predictors], target)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features=3, max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=128, n_jobs=6,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [9]:
from tabulate import tabulate
headers = ["name", "score"]
values = sorted(zip(X_test.columns, full_model.feature_importances_), key=lambda x: x[1] * -1)
print(tabulate(values, headers, tablefmt="plain"))

name                       score
target_lag_1           0.240744
item_target_enc_ave    0.238609
item_target_enc_lag_1  0.13398
target_lag_2           0.119803
shop_target_enc_lag_1  0.0875946
target_lag_3           0.0733154
shop_target_enc_ave    0.0490375
target_lag_6           0.0380415
target_lag_12          0.0188757


In [10]:
%store -r __prepare_submission

In [11]:
__prepare_submission

In [12]:
aliases = \
{
    "target_lag_1":          "target",
    "target_lag_2":          "target_lag_1",
    "target_lag_3":          "target_lag_2",
    "target_lag_6":          "target_lag_5",
    "target_lag_12":         "target_lag_11",
    "item_target_enc_lag_1": "item_target_enc",
    "shop_target_enc_lag_1": "shop_target_enc",
    "item_target_enc_ave":   "item_target_enc_ave",
    "shop_target_enc_ave":   "shop_target_enc_ave"
 }

prepare_submission(full_model, predictors, aliases, "smarter_lags.csv")